<a href="https://colab.research.google.com/github/Yaaxek/-Pandas-seleccionar-y-agrupar-datos/blob/main/Pandas_Impacto_de_los_GEI_en_Brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Conociendo los datos**

Analizaremos datos de emisiones de gases de efecto invernadero en Brasil y crearemos tablas y visualizaciones para comprender los datos y generar insights a partir de ellos. Los datos fueron extraídos del SEEG.

---
ℹ Fuente: SEEG - Sistema de Estimativa de Emissões e Remoções de Gases de Efeito Estufa, Observatorio de Clima, accesado en http://seeg.eco.br:

- [Download de la base de datos](http://seeg.eco.br/download)
---

Primero, vamos a importar la biblioteca [`pandas`](https://pandas.pydata.org/docs/index.html) y los datos que serán utilizados. Como los datos están en un archivo excel, vamos a usar la función [`read_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html).

##Leyendo la base de datos

In [ ]:
import pandas as pd

In [ ]:
emisiones_gases=pd.read_excel('1-SEEG10_GERAL-BR_UF_2022.10.27-FINAL-SITE.xlsx', sheet_name='GEE Estados')

In [ ]:
emisiones_gases.sample(5)

Vamos a revisar la estructura con el metodo **info()**

In [ ]:
emisiones_gases.info()

## Ajustando la base datos

Solo nos interesan las emisiones de gases fenerados por los estados que estan en el inventario nacional, por lo que identificaremos los tipos de datos unicos en la columna Emissão / Remoção / Bunker

Remoção:datos referentes a la eliminacion de gases, lo contrario a las emisiones

NCI: son datos no incluidos en el inventario nacional.

Bunker: emisiones del transporte maritimo y aereo internacional, no son emisiones direcas de los estado.

In [ ]:
emisiones_gases['Emissão / Remoção / Bunker'].unique()

Se analizara primero los datos de remoção (eliminacion), para identificar si todos ellos son de retirada de gases de efecto invernadero.

In [ ]:
#Se revisara que los datos esten correctamente asignado
(emisiones_gases['Emissão / Remoção / Bunker']=='Remoção NCI') | (emisiones_gases['Emissão / Remoção / Bunker']=='Remoção')

In [ ]:
emisiones_gases[emisiones_gases['Emissão / Remoção / Bunker'].isin(['Remoção NCI','Remoção'])]

In [ ]:
emisiones_gases.loc[emisiones_gases['Emissão / Remoção / Bunker'].isin(['Remoção NCI','Remoção']),1970:2021]

Para verificar si todos son negativos, vamos a utiliza la funcion **max()**, si alguno es mayor que 0 esta mal clasificado deberia de estar en emision

In [ ]:
emisiones_gases.loc[emisiones_gases['Emissão / Remoção / Bunker'].isin(['Remoção NCI','Remoção']),1970:2021].max()

Ya se verifico que si esta en remocion entonces se podran eliminar

Se analizara ahora Bunker para verificar que si es fuera del Estado

In [ ]:
emisiones_gases.loc[emisiones_gases['Emissão / Remoção / Bunker']=='Bunker', 'Estado'].unique()

Ya se verifico que fue en aguas internacionales, asi que tambien se puede borrar

In [ ]:
emision_gases=emisiones_gases[emisiones_gases['Emissão / Remoção / Bunker']=='Emissão']
emision_gases

In [ ]:
emision_gases=emision_gases.drop(columns='Emissão / Remoção / Bunker')
emision_gases

# 2. Agrupamiento de los datos

##Modificando el formato del DataFrame

Transformar de wide a long

In [ ]:
import pandas as pd

In [ ]:
emision_gases.loc[:,'Nível 1 - Setor':'Produto'].columns

In [ ]:
columnas_info=list(emision_gases.loc[:,'Nível 1 - Setor':'Produto'].columns)
columnas_info

In [ ]:
emisiones_gases.loc[:,1970:2021].columns

In [ ]:
columnas_emision=list(emisiones_gases.loc[:,1970:2021].columns)
columnas_emision

Para modificar vamos a usar el metodo **melt()** para pasar de wide a long

id_vars: lo que quiero conservar

value_vars: lo que quiero modificar

var_name:nombre de lo modificado

value_name: donde se guardan los nuevos valores

In [ ]:
emisiones_por_ano=emision_gases.melt(id_vars=columnas_info, value_vars=columnas_emision, var_name='Año', value_name='Emision')
emisiones_por_ano

#Analisis de las emisiones

In [ ]:
emisiones_por_ano.groupby('Gás')

In [ ]:
emisiones_por_ano.groupby('Gás').groups

In [ ]:
emisiones_por_ano.groupby('Gás').get_group('CO2 (t)')

In [ ]:
emisiones_por_ano.groupby('Gás').sum()

#**Desafio**

Encuentre los valores únicos de las columnas "Nível 1 - Setor" y "Estado" para identificar las actividades económicas presentes en la base de datos y si todos los estados brasileños están presentes en el DataFrame.



In [ ]:
emisiones_gases['Nível 1 - Setor'].unique()

In [ ]:
emisiones_gases['Estado'].unique()

Filtrar el DataFrame sólo con datos de estados de la región sur de Brasil.



In [ ]:
sur=emisiones_gases[emisiones_gases['Estado'].isin(['PR','RS','SC'])]
sur


Filtrar el DataFrame sólo con datos de "Mudança de Uso da Terra e Floresta" que sean del Estado de Amazonas.



In [ ]:
amazonas=emisiones_gases[(emisiones_gases['Nível 1 - Setor']=='Mudança de Uso da Terra e Floresta') & (emisiones_gases['Estado']=='AM')]
amazonas

Encuentre el valor máximo de emisión para el año 2021 para los datos de "Agropecuária" en el Estado de Pará.



In [ ]:
agropecuaria=emision_gases.loc[(emision_gases['Nível 1 - Setor'] == 'Agropecuária') & (emision_gases['Estado'] == 'PA'), 2021].max()
agropecuaria